In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
import os

In [11]:
url_serie_a        = "https://fbref.com/en/comps/11/schedule/Serie-A-Scores-and-Fixtures"
url_ligue_1        = "https://fbref.com/en/comps/13/schedule/Ligue-1-Scores-and-Fixtures"
url_bundesliga     = "https://fbref.com/en/comps/20/schedule/Bundesliga-Scores-and-Fixtures"
url_premier_league = "https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures"
url_la_liga        = "https://fbref.com/en/comps/12/schedule/La-Liga-Scores-and-Fixtures"
url_ldc            = "https://fbref.com/en/comps/8/schedule/Champions-League-Scores-and-Fixtures"
url_el             = "https://fbref.com/en/comps/19/schedule/Europa-League-Scores-and-Fixtures"
url_ecl            = "https://fbref.com/en/comps/882/schedule/Conference-League-Scores-and-Fixtures"

url_leagues     = [url_serie_a, url_ligue_1, url_bundesliga, url_premier_league, url_la_liga, url_ldc, url_el, url_ecl]
url_id_tab_list = ["sched_2024-2025_11_1", "sched_2024-2025_13_1", "sched_2024-2025_20_1", "sched_2024-2025_9_1", "sched_2024-2025_12_1", "sched_all", "sched_all", "sched_all"]

leagues         = ["Serie A", "Ligue 1", "Bundesliga", "Premier League", "La Liga", "Champions League", "Europa League", "Europa Conference League"]

path_folder     = "/Users/matteolemesre/Desktop/Data LOSC/csv/csv24_25/"

In [2]:
def new_gameweek(gameweek, count):
    if gameweek == 'Knockout phase play-offs' and count <= 8:
        gameweek = 'J9'
        count += 1
    elif gameweek == 'Knockout phase play-offs' and count <= 16:
        gameweek = 'J10'
        count += 1
    elif gameweek == 'Round of 16' and count <= 24:
        gameweek = 'J11'
        count += 1
    elif gameweek == 'Round of 16' and count <= 32:
        gameweek = 'J12'
        count += 1
    elif gameweek == 'Quarter-finals' and count <= 36:
        gameweek = 'J13'
        count += 1
    elif gameweek == 'Quarter-finals' and count <= 40:
        gameweek = 'J14'
        count += 1
    elif gameweek == 'Semi-finals' and count <= 42:
        gameweek = 'J15'
        count += 1
    elif gameweek == 'Semi-finals' and count <= 44:
        gameweek = 'J16'
        count += 1
    if gameweek == 'Final':
        gameweek = 'J17'
        count += 1
    return gameweek, count

In [3]:
def data_game(url_game, game_week, home_team, away_team, league):
    tables = pd.read_html(url_game)
    
    players_home = pd.concat(tables[3:9], axis=1)
    players_home = players_home[:-2]
    players_home.insert(0, "Game Week", game_week)  
    players_home.insert(2, "Team", home_team)   
    players_home.insert(3, "League", league)   
    
    players_away = pd.concat(tables[10:16], axis=1)
    players_away = players_away[:-2]
    players_away.insert(0, "Game Week", game_week)  
    players_away.insert(2, "Team", away_team)
    players_away.insert(3, "League", league)      
    
    data_players = pd.concat([players_home, players_away], axis=0)
    data_players.reset_index(drop=True, inplace=True)
    data_players.columns = data_players.columns.droplevel(0)
    
    goals_home = tables[9]
    goals_home.insert(0, "Game Week", game_week)  
    goals_home.insert(2, "Team", home_team)  
    goals_home.insert(3, "League", league)   
    
    goals_away = tables[16]
    goals_away.insert(0, "Game Week", game_week)  
    goals_away.insert(2, "Team", away_team)
    goals_away.insert(3, "League", league)       
    
    data_goals = pd.concat([goals_home, goals_away], axis=0)
    data_goals.reset_index(drop=True, inplace=True)
    data_goals.columns = data_goals.columns.droplevel(0)
    
    cols = list(data_players.columns)
    cols[0] = "Game Week"
    cols[2] = "Team"
    cols[3] = "League"
    data_players.columns = cols

    cols = list(data_goals.columns)
    cols[0] = "Game Week"
    cols[2] = "Team"
    cols[3] = "League"
    data_goals.columns = cols

    
    return data_players, data_goals
    

In [4]:
def url_games_countries(url_league, url_id_table):
    options = Options()
    options.add_argument("--headless")
    service = Service("/Users/matteolemesre/Desktop/chromedriver2/chromedriver")
    driver = webdriver.Chrome(service=service, options=options)
    
    driver.get(url_league)
    time.sleep(5)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    table = soup.find("table", id=url_id_table)

    data_games_list = []

    if table:
        rows = table.find_all("tr", attrs={"data-row": True})
        for row in rows:
            week = row.find("th", {"data-stat": "gameweek"})
            date = row.find("td", {"data-stat": "date"})
            home = row.find("td", {"data-stat": "home_team"})
            away = row.find("td", {"data-stat": "away_team"})
            score = row.find("td", {"data-stat": "score"})
            attendance = row.find("td", {"data-stat": "attendance"})
            venue = row.find("td", {"data-stat": "venue"})
            referee = row.find("td", {"data-stat": "referee"})
            report = row.find("td", {"data-stat": "match_report"})

            if week and home and away:
                game_week = f"J{week.text.strip()}"
                home_team = home.text.strip()
                away_team = away.text.strip()
                link = None

                if report:
                    a = report.find("a")
                    if a and a.has_attr("href"):
                        link = "https://fbref.com" + a["href"]

                data_games_list.append({
                    "Game Week": game_week,
                    "Home Team": home_team,
                    "Away Team": away_team,
                    "Score": score.text.strip() if score else None,
                    "Attendance": attendance.text.strip() if attendance else None,
                    "Venue": venue.text.strip() if venue else None,
                    "Referee": referee.text.strip() if referee else None,
                    "Date": date.text.strip() if date else None,
                    "URL": link
                })
                
    data_games_list = data_games_list.dropna(subset=["Home Team", "Away Team", "Score"])   
        
    return data_games_list

In [5]:
def url_games_europe(url_league, url_id_table):
    options = Options()
    options.add_argument("--headless")
    service = Service("/Users/matteolemesre/Desktop/chromedriver2/chromedriver")
    driver = webdriver.Chrome(service=service, options=options)

    driver.get(url_league)
    time.sleep(5)  

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    table = soup.find("table", id=url_id_table)

    data_games_list = []
    rows = table.find_all("tr", attrs={"data-row": True})

    count = 1
    for row in rows:
        week = row.find("th", {"data-stat": "round"})
        if week.text.strip() == "League phase":
            week = row.find("td", {"data-stat": "gameweek"})
            game_week = "J" + week.text.strip() 
        else:
            game_week = week.text.strip()
        date = row.find("td", {"data-stat": "date"})
        home = row.find("td", {"data-stat": "home_team"})
        if home:
            home_team_text = home.text.strip()
            home_team_words = home_team_text.split()
            home = " ".join(home_team_words[:-1]) if len(home_team_words) > 1 else home_team_text
        away = row.find("td", {"data-stat": "away_team"})
        if away:
            away_team_text = away.text.strip()
            away_team_words = away_team_text.split()
            away = " ".join(away_team_words[1:]) if len(away_team_words) > 1 else away_team_text
        score = row.find("td", {"data-stat": "score"})
        attendance = row.find("td", {"data-stat": "attendance"})
        venue = row.find("td", {"data-stat": "venue"})
        referee = row.find("td", {"data-stat": "referee"})
        report = row.find("td", {"data-stat": "match_report"})

        if game_week and home and away:
            game_week, count = new_gameweek(game_week, count)
            link = None

            if report:
                a = report.find("a")
                if a and a.has_attr("href"):
                    link = "https://fbref.com" + a["href"]

            data_games_list.append({
                "Game Week": game_week,
                "Home Team": home,
                "Away Team": away,
                "Score": score.text.strip() if score else None,
                "Attendance": attendance.text.strip() if attendance else None,
                "Venue": venue.text.strip() if venue else None,
                "Referee": referee.text.strip() if referee else None,
                "Date": date.text.strip() if date else None,
                "URL": link
            })
    
    data_games_list = data_games_list.dropna(subset=["Home Team", "Away Team", "Score"])
            
    return data_games_list

In [6]:
def url_games(url_league, url_id_table):
    if url_id_table == "sched_all":
        data_games_list = url_games_europe(url_league, url_id_table)
    else:
        data_games_list = url_games_countries(url_league, url_id_table)
    return data_games_list

In [6]:
new_column_names_players = [
    "Game Week",  # Semaine de jeu
    "Player",  # Nom du joueur
    "Team",  # Équipe
    "League",  # Ligue
    "Shirt Number",  # Numéro de maillot
    "Nationality",  # Nationalité
    "Position",  # Poste
    "Age",  # Âge
    "Minutes",  # Minutes jouées
    "Goals",  # Buts
    "Assists",  # Passes décisives
    "Penalty Kicks Made",  # Pénaltys marqués
    "Penalty Kicks Attempted",  # Pénaltys tentés
    "Shots Total",  # Tirs totaux (hors pénaltys)
    "Shots on Target",  # Tirs cadrés
    "Yellow Cards",  # Cartons jaunes
    "Red Cards",  # Cartons rouges
    "Touches",  # Touches de balle
    "Tackles",  # Tacles réussis
    "Interceptions",  # Interceptions
    "Blocks",  # Contres
    "Expected Goals (xG)",  # Buts attendus (xG)
    "Non-Penalty Expected Goals (npxG)",  # xG hors pénaltys
    "Expected Assisted Goals (xAG)",  # xA - passes décisives attendues
    "Shot-Creating Actions (SCA)",  # Actions menant à un tir
    "Goal-Creating Actions (GCA)",  # Actions menant à un but
    "Passes Completed",  # Passes réussies
    "Passes Attempted",  # Passes tentées
    "Pass Completion %",  # % de réussite des passes
    "Progressive Passes",  # Passes progressives
    "Carries",  # Conduites de balle
    "Progressive Carries",  # Conduites de balle progressives
    "Take-Ons Attempted",  # Dribbles tentés
    "Successful Take-Ons",  # Dribbles réussis
    "Player",  # Nom du joueur
    "Shirt Number",  # Numéro de maillot
    "Nationality",  # Nationalité
    "Position",  # Poste
    "Age",  # Âge
    "Minutes Played",  # Minutes jouées
    "Passes Completed (Total)",  # Passes réussies - total
    "Passes Attempted (Total)",  # Passes tentées - total
    "Pass Completion % (Total)",  # % de réussite - total
    "Total Passing Distance",  # Distance totale parcourue par les passes
    "Progressive Passing Distance",  # Distance progressive des passes
    "Passes Completed (Short)",  # Passes réussies - courtes
    "Passes Attempted (Short)",  # Passes tentées - courtes
    "Pass Completion % (Short)",  # % de réussite - courtes
    "Passes Completed (Medium)",  # Passes réussies - moyennes
    "Passes Attempted (Medium)",  # Passes tentées - moyennes
    "Pass Completion % (Medium)",  # % de réussite - moyennes
    "Passes Completed (Long)",  # Passes réussies - longues
    "Passes Attempted (Long)",  # Passes tentées - longues
    "Pass Completion % (Long)",  # % de réussite - longues
    "Assists",  # Passes décisives
    "Expected Assisted Goals (xAG)",  # xA généré par les passes
    "Expected Assists (xA)",  # xA global
    "Key Passes",  # Passes clés (menant à un tir)
    "Passes into Final Third",  # Passes dans le dernier tiers
    "Passes into Penalty Area",  # Passes dans la surface
    "Crosses into Penalty Area",  # Centres dans la surface
    "Progressive Passes",  # Passes progressives
    "Player",  # Nom du joueur
    "Shirt Number",  # Numéro de maillot
    "Nationality",  # Nationalité
    "Position",  # Poste
    "Age",  # Âge
    "Minutes Played",  # Minutes jouées
    "Passes Attempted",  # Passes tentées
    "Live-ball Passes",  # Passes en jeu
    "Dead-ball Passes",  # Passes arrêtées
    "Passes from Free Kicks",  # Passes sur coup franc
    "Through Balls",  # Passes en profondeur
    "Switches",  # Changements d’aile
    "Crosses",  # Centres
    "Throw-ins Taken",  # Touches jouées
    "Corner Kicks",  # Corners totaux
    "Inswinging Corner Kicks",  # Corners rentrants
    "Outswinging Corner Kicks",  # Corners sortants
    "Straight Corner Kicks",  # Corners droits
    "Passes Completed",  # Passes réussies
    "Passes Offside",  # Passes provoquant un hors-jeu
    "Passes Blocked",  # Passes contrées
    "Player",  # Nom du joueur
    "Shirt Number",  # Numéro de maillot
    "Nationality",  # Nationalité
    "Position",  # Poste
    "Age",  # Âge
    "Minutes Played",  # Minutes jouées
    "Tackles",  # Tacles total
    "Tackles Won",  # Tacles gagnés (avec récupération)
    "Tackles in Defensive Third",  # Tacles dans le tiers défensif
    "Tackles in Middle Third",  # Tacles dans le tiers du milieu
    "Tackles in Attacking Third",  # Tacles dans le tiers offensif
    "Dribblers Tackled",  # Dribbleurs taclés
    "Dribbles Challenged",  # Dribbles défendus (tacles + ratés)
    "Tackle Success % vs Dribblers",  # % de réussite contre les dribbleurs
    "Challenges Lost",  # Défenses ratées sur dribbleur
    "Blocks",  # Ballons bloqués
    "Shots Blocked",  # Tirs bloqués
    "Passes Blocked",  # Passes bloquées
    "Interceptions",  # Interceptions
    "Tackles + Interceptions",  # Tacles + interceptions
    "Clearances",  # Dégagements
    "Errors Leading to Shot",  # Erreurs menant à un tir
    "Player",  # Nom du joueur
    "Shirt Number",  # Numéro de maillot
    "Nationality",  # Nationalité
    "Position",  # Poste
    "Age",  # Âge
    "Minutes Played",  # Minutes jouées
    "Touches",  # Touches totales
    "Touches in Defensive Penalty Area",  # Dans la surface défensive
    "Touches in Defensive Third",  # Dans le tiers défensif
    "Touches in Middle Third",  # Dans le tiers central
    "Touches in Attacking Third",  # Dans le tiers offensif
    "Touches in Attacking Penalty Area",  # Dans la surface offensive
    "Live-Ball Touches",  # Touches en jeu
    "Take-Ons Attempted",  # Dribbles tentés
    "Take-Ons Succeeded",  # Dribbles réussis
    "Take-On Success %",  # % de réussite en dribble
    "Times Tackled During Take-On",  # Tacles subis en dribble
    "Tackled During Take-On %",  # % de dribbles stoppés
    "Carries",  # Conduites de balle
    "Total Carrying Distance",  # Distance totale parcourue avec le ballon
    "Progressive Carrying Distance",  # Distance progressive
    "Progressive Carries",  # Conduites progressives
    "Carries into Final Third",  # Conduites dans le dernier tiers
    "Carries into Penalty Area",  # Conduites dans la surface
    "Miscontrols",  # Contrôles ratés
    "Dispossessed",  # Ballons perdus (dépossédé)
    "Passes Received",  # Passes reçues
    "Progressive Passes Received",  # Passes progressives reçues
    "Player",  # Nom du joueur
    "Shirt Number",  # Numéro de maillot
    "Nationality",  # Nationalité
    "Position",  # Poste
    "Age",  # Âge
    "Minutes Played",  # Minutes jouées
    "Yellow Cards",  # Cartons jaunes
    "Red Cards",  # Cartons rouges
    "Second Yellow Card",  # Deuxième jaune
    "Fouls Committed",  # Fautes commises
    "Fouls Drawn",  # Fautes subies
    "Offsides",  # Hors-jeu
    "Crosses",  # Centres
    "Interceptions",  # Interceptions
    "Tackles Won",  # Tacles réussis
    "Penalty Kicks Won",  # Penalties obtenus
    "Penalty Kicks Conceded",  # Penalties concédés
    "Own Goals",  # Buts contre son camp
    "Ball Recoveries",  # Ballons récupérés
    "Aerials Won",  # Duels aériens gagnés
    "Aerials Lost",  # Duels aériens perdus
    "Aerial Win %"  # % de duels aériens gagnés
]


In [13]:
new_column_names_goals = [
    "Game Week",  # Semaine de jeu
    "Player",  # Nom du joueur
    "Team",  # Équipe
    "League",  # Ligue
    "Nationality",  # Nationalité
    "Age",  # Âge
    "Minutes",  # Minutes jouées
    "Shots on Target Against",  # Tirs cadrés subis
    "Goals Against",  # Buts encaissés
    "Saves",  # Arrêts
    "Save Percentage",  # Pourcentage d'arrêts
    "Post-Shot Expected Goals (PSxG)",  # PSxG - xG après tir (qualité des tirs cadrés subis)
    "Passes Completed (Launched)",  # Passes longues réussies (> 40 yards)
    "Passes Attempted (Launched)",  # Passes longues tentées
    "Pass Completion % (Launched)",  # % de passes longues réussies
    "Passes Attempted (GK)",  # Passes tentées (hors goal kicks)
    "Throws Attempted",  # Lancers
    "Launch % (Passes)",  # % de passes longues parmi les passes hors goal kicks
    "Average Pass Length",  # Longueur moyenne des passes
    "Goal Kicks Attempted",  # Dégagements au pied
    "Launch % (Goal Kicks)",  # % de dégagements longs
    "Average Goal Kick Length",  # Longueur moyenne des dégagements
    "Crosses Faced",  # Centres adverses subis
    "Crosses Stopped",  # Centres interceptés
    "Cross Stop %",  # % de centres interceptés
    "Def. Actions Outside Penalty Area",  # Actions défensives hors de la surface
    "Average Distance of Def. Actions"  # Distance moyenne de ces actions
]

In [8]:
def players_column_names(df, new_column_names_players):
    df.columns = new_column_names_players
    return df

In [14]:
def goals_column_names(df, new_column_names_goals):
    df.columns = new_column_names_goals
    return df

In [10]:
for i in range(len(url_leagues)):
    url_league = url_leagues[i]
    url_id_table = url_id_tab_list[i]
    league = leagues[i]
    print(f"In {league}:")
    data_games_list = url_games(url_league, url_id_table)
    print(f"→ {len(data_games_list)} matches found in {league}.")
    pd.DataFrame(data_games_list).to_csv(os.path.join(path_folder, f"leagues_games/{league}_games.csv"), index=False)


For Serie A:
→ 420 matchs trouvés dans Serie A.
For Ligue 1:
→ 340 matchs trouvés dans Ligue 1.
For Bundesliga:
→ 339 matchs trouvés dans Bundesliga.
For Premier League:
→ 420 matchs trouvés dans Premier League.
For La Liga:
→ 426 matchs trouvés dans La Liga.
For Champions League:
→ 189 matchs trouvés dans Champions League.
For Europa League:
→ 189 matchs trouvés dans Europa League.
For Europa Conference League:
→ 153 matchs trouvés dans Europa Conference League.


In [11]:
no_games = 1
for i in range(len(url_leagues)):
    url_league = url_leagues[i]
    url_id_table = url_id_tab_list[i]
    league = leagues[i]
    print(f"For {league}:")
    data_games_list = url_games(url_league, url_id_table)
    print(f"→ {len(data_games_list)} matches found in {league}.")
    pd.DataFrame(data_games_list).to_csv(os.path.join(path_folder, f"leagues_games/{league}_games.csv"), index=False)
    for data_games in data_games_list:
        game_week = data_games["Game Week"]
        home_team = data_games["Home Team"]
        away_team = data_games["Away Team"]
        url_game = data_games["URL"]
        time.sleep(random.uniform(2, 4))

        try:
            data_players, data_goals = data_game(
                url_game,
                game_week,
                home_team,
                away_team,
                league
            )
        except Exception as e:
            print(f"⚠️ Failed for URL {url_game}: {e}")
            continue
        
        print(no_games)
        data_players.to_csv(os.path.join(path_folder, f"raw data/raw data bis/data_players_{no_games}.csv"), index=False)
        data_goals.to_csv(os.path.join(path_folder, f"raw data/raw data bis/data_goals_{no_games}.csv"), index=False)
        no_games += 1

For Serie A:
→ 420 matchs trouvés dans Serie A.
1
2
3
4
5
6
7
8
9
10
⚠️ Échec pour l’URL None : cannot parse from 'NoneType'
11
12
13
14
15
16
17
18
19
20
⚠️ Échec pour l’URL None : cannot parse from 'NoneType'
21
22
23
24
25
26
27
28
29
30
⚠️ Échec pour l’URL None : cannot parse from 'NoneType'
31
32
33
34
35
36
37
38
39
40
⚠️ Échec pour l’URL None : cannot parse from 'NoneType'
41
42
43
44
45
46
47
48
49
50
⚠️ Échec pour l’URL None : cannot parse from 'NoneType'
51
52
53
54
55
56
57
58
59
60
⚠️ Échec pour l’URL None : cannot parse from 'NoneType'
61
62
63
64
65
66
67
68
69
70
⚠️ Échec pour l’URL None : cannot parse from 'NoneType'
71
72
73
74
75
76
77
78
79
80
⚠️ Échec pour l’URL None : cannot parse from 'NoneType'
81
82
83
84
85
86
87
88
89
⚠️ Échec pour l’URL None : cannot parse from 'NoneType'
90
91
92
93
94
95
96
97
98
99
⚠️ Échec pour l’URL None : cannot parse from 'NoneType'
100
101
102
103
104
105
106
107
108
109
⚠️ Échec pour l’URL None : cannot parse from 'NoneType'
110
111


In [15]:
folder_path = os.path.join(path_folder, "brutes/brutes bis/")

data_players_list = []
data_goals_list = []

player_files = sorted([f for f in os.listdir(folder_path) if f.startswith("data_players_") and f.endswith(".csv")])
goal_files = sorted([f for f in os.listdir(folder_path) if f.startswith("data_goals_") and f.endswith(".csv")])

for file_name in player_files:
    file_path = os.path.join(folder_path, file_name)
    data_players_list.append(pd.read_csv(file_path))

for file_name in goal_files:
    file_path = os.path.join(folder_path, file_name)
    data_goals_list.append(pd.read_csv(file_path))

df_players = pd.concat(data_players_list, ignore_index=True)
df_goals = pd.concat(data_goals_list, ignore_index=True)

df_players = players_column_names(df_players, new_column_names_players)
df_goals = goals_column_names(df_goals, new_column_names_goals)

df_players.to_csv(os.path.join(path_folder, "brutes/data_players.csv"), index=False)
df_goals.to_csv(os.path.join(path_folder, "brutes/data_goals.csv"), index=False)